# Merge data sources into database

In [1]:
import os
import pandas as pd
import numpy as np

## Game scores (regular season and playoffs):
CSV data copied from: <br>
https://www.hockey-reference.com/leagues/NHL_2019_games.html

In [2]:
dfs = []
for f in os.listdir():
    if f.endswith('.csv'):
        name = f[:-4].split('_')
        season = '20' + name[1] + '-20' + name[2]
        category = name[3] + ' ' + name[4] if len(name) > 4 else name[3]
        tmp_df = pd.read_csv(f)
        tmp_df['Season'] = season
        tmp_df['Type'] = category
        dfs.append(tmp_df)
scores = pd.concat(dfs)

In [3]:
print(scores['Season'].unique())

['2018-2019' '2020-2021' '2019-2020']


In [4]:
scores.head()

,Date,Visitor,G,Home,G.1,Unnamed: 5,Att.,LOG,Notes,Season,Type
0,2019-04-10,Pittsburgh Penguins,3.0,New York Islanders,4.0,OT,13917.0,3:06,at Nassau Veterans Memorial Coliseum (Uniondal...,2018-2019,playoffs
1,2019-04-10,Dallas Stars,3.0,Nashville Predators,2.0,NaN,17458.0,2:30,NaN,2018-2019,playoffs
2,2019-04-10,St. Louis Blues,2.0,Winnipeg Jets,1.0,NaN,15321.0,2:31,NaN,2018-2019,playoffs
3,2019-04-10,Vegas Golden Knights,2.0,San Jose Sharks,5.0,NaN,17562.0,2:39,NaN,2018-2019,playoffs
4,2019-04-10,Columbus Blue Jackets,4.0,Tampa Bay Lightning,3.0,NaN,19092.0,2:28,NaN,2018-2019,playoffs


Let's change some of the column names.<br>
  * Goals should be clear
  * Unnamed shows Overtime
  * Att. stands for Attendence
  * LOG stands for Length of Game

In [5]:
scores = scores.rename(columns={'G':'Visitor Goals', 'G.1':'Home Goals', 'Unnamed: 5':'Overtime', 'Att.':'Attendance', 'LOG':'Length'}, inplace=False)

Let's remove the data for the games that haven't been played yet.

In [6]:
scores.loc[scores['Visitor Goals'].isnull()]

,Date,Visitor,Visitor Goals,Home,Home Goals,Overtime,Attendance,Length,Notes,Season,Type
692,2021-04-20,Boston Bruins,NaN,Buffalo Sabres,NaN,NaN,NaN,NaN,NaN,2020-2021,regular season
693,2021-04-20,Detroit Red Wings,NaN,Dallas Stars,NaN,NaN,NaN,NaN,NaN,2020-2021,regular season
694,2021-04-20,Columbus Blue Jackets,NaN,Florida Panthers,NaN,NaN,NaN,NaN,NaN,2020-2021,regular season
695,2021-04-20,Anaheim Ducks,NaN,Los Angeles Kings,NaN,NaN,NaN,NaN,NaN,2020-2021,regular season
696,2021-04-20,New York Rangers,NaN,New York Islanders,NaN,NaN,NaN,NaN,at Nassau Veterans Memorial Coliseum (Uniondal...,2020-2021,regular season
...,...,...,...,...,...,...,...,...,...,...,...
860,2021-05-14,Toronto Maple Leafs,NaN,Winnipeg Jets,NaN,NaN,NaN,NaN,NaN,2020-2021,regular season
861,2021-05-15,Vancouver Canucks,NaN,Edmonton Oilers,NaN,NaN,NaN,NaN,NaN,2020-2021,regular season
862,2021-05-16,Calgary Flames,NaN,Vancouver Canucks,NaN,NaN,NaN,NaN,NaN,2020-2021,regular season
863,2021-05-18,Calgary Flames,NaN,Vancouver Canucks,NaN,NaN,NaN,NaN,NaN,2020-2021,regular season


In [7]:
scores = scores.dropna(subset=['Visitor Goals'])
scores['Attendance'] = scores['Attendance'].fillna(0)
scores['Length'] = scores['Length'].fillna('0:00')

Any convert the data types, just to clean things up a bit.

In [8]:
scores.dtypes

Date              object
Visitor           object
Visitor Goals    float64
Home              object
Home Goals       float64
Overtime          object
Attendance       float64
Length            object
Notes             object
Season            object
Type              object
dtype: object

In [9]:
scores.memory_usage(index=False)

Date             26096
Visitor          26096
Visitor Goals    26096
Home             26096
Home Goals       26096
Overtime         26096
Attendance       26096
Length           26096
Notes            26096
Season           26096
Type             26096
dtype: int64

In [10]:
scores['Date'] = scores['Date'].astype('datetime64')
scores['Visitor'] = scores['Visitor'].astype('category')
scores['Visitor Goals'] = scores['Visitor Goals'].astype('int64')
scores['Home'] = scores['Home'].astype('category')
scores['Home Goals'] = scores['Home Goals'].astype('int64')
scores['Overtime'] = scores['Overtime'].astype('category')
scores['Attendance'] = scores['Attendance'].astype('int64')
scores['Length'] = pd.to_timedelta((scores['Length'].str.split(':', expand=True).astype(int) * (60, 1)).sum(axis=1), unit='min')
scores['Season'] = scores['Season'].astype('category')
scores['Type'] = scores['Type'].astype('category')

In [11]:
scores.head()

,Date,Visitor,Visitor Goals,Home,Home Goals,Overtime,Attendance,Length,Notes,Season,Type
0,2019-04-10,Pittsburgh Penguins,3,New York Islanders,4,OT,13917,03:06:00,at Nassau Veterans Memorial Coliseum (Uniondal...,2018-2019,playoffs
1,2019-04-10,Dallas Stars,3,Nashville Predators,2,NaN,17458,02:30:00,NaN,2018-2019,playoffs
2,2019-04-10,St. Louis Blues,2,Winnipeg Jets,1,NaN,15321,02:31:00,NaN,2018-2019,playoffs
3,2019-04-10,Vegas Golden Knights,2,San Jose Sharks,5,NaN,17562,02:39:00,NaN,2018-2019,playoffs
4,2019-04-10,Columbus Blue Jackets,4,Tampa Bay Lightning,3,NaN,19092,02:28:00,NaN,2018-2019,playoffs


Let's save the data for now.

In [12]:
scores.to_pickle('scores.pkl')